In [71]:
import numpy as np
import matplotlib.pyplot as plt
import open3d
from open3d import utility
import cv2
import glob
import math
from tqdm import tqdm

In [73]:
Imgs = []
for file in glob.glob('Data/CameraModel/images/ho*'):
    Imgs.append(cv2.imread(file)[:,:,0])
Imgs = np.array(Imgs)

file = open('Data/CameraModel/2D/house.000.corners', 'r')
points1 = np.array([ np.array(st.split()).astype(float) for st in file.read().split('\n')[:-1] ]).astype(int)
#points1 = np.hstack((points1, np.ones((points1.shape[0], 1))))
file = open('Data/CameraModel/2D/house.001.corners', 'r')
points2 = np.array([ np.array(st.split()).astype(float) for st in file.read().split('\n')[:-1] ]).astype(int)
#points2 = np.hstack((points2, np.ones((points2.shape[0], 1))))
file = open('Data/CameraModel/2D/house.002.corners', 'r')
points3 = np.array([ np.array(st.split()).astype(float) for st in file.read().split('\n')[:-1] ]).astype(int)
#points3 = np.hstack((points3, np.ones((points3.shape[0], 1))))

file = open('Data/CameraModel/2D/house.nview-corners', 'r')
PntsMatch = np.array([ st.split() for st in file.read().split('  \n')[:-1] ])[:,:3]
PntsMatch[PntsMatch == '*'] = '-1'
PntsMatch = PntsMatch.astype(int)
PntsMatch = PntsMatch[((PntsMatch[:,0] != -1) & (PntsMatch[:,1] != -1)) | ((PntsMatch[:,2] != -1) & (PntsMatch[:,0] != -1)) | ((PntsMatch[:,1] != -1) & (PntsMatch[:,2] != -1))]

IntPnts = [points1, points2, points3]
#matches = (match_table[:,0] != '*') & (match_table[:,1] != '*')
#matches = match_table[matches].astype(int)

In [74]:
class Reconstruction():
    
    def __init__(self, img_adds, param_dict='default'):
        self.Imgs = []
        for file in img_adds:
            img = cv2.imread(file)
            self.Imgs.append(img)
        self.Imgs = np.array(self.Imgs)
        print(f'Number of Input Images: {self.Imgs.shape[0]}')
        if len(self.Imgs[0].shape) == 3:
            self.H, self.W, self.channels = self.Imgs[0].shape
        else:
            (self.H, self.W), self.channels = self.Imgs[0].shape, 1
        
        if param_dict == 'default':
            self.param_dict = {}
            self.param_dict['loop_F_mat'] = 200
            self.param_dict['thld_F_inliers'] = 0.2
            self.param_dict['thld_P2_angle'] = 70
            self.param_dict['thld_zncc_seed'] = 0.9
            self.param_dict['thld_confidence'] = 0.01
            self.param_dict['w_seed'] = 5
            self.param_dict['thld_zncc_propagation'] = 0.9
            self.param_dict['w_propagation'] = 2
            self.param_dict['N_window'] = 2
            self.param_dict['epsilon_propagation'] = 1
            
    def get_match_pair_list(self, i, j):
        points1 = self.PntsMatch[:,i]
        points2 = self.PntsMatch[:,j]
        true_index = (points1 != -1) and (points2 != -1)
        points1 = self.IntPnts[i][points1[true_index]]
        points2 = self.IntPnts[j][points2[true_index]]
        return np.hstack((points1, np.ones(points1.shape[0]).T)), np.hstack((points2, np.ones(points2.shape[0]).T))
    
    def triangulate(self, p1, p2, K, P1, P2):
        
        Q = np.zeros((6, 6))
        Q[:3, :4] = np.matmul(K, P1)
        Q[3:, :4] = np.matmul(K, P2)
        Q[:3, 4] = -p1
        Q[3:, 5] = -p2
        
        U, S, Vt = np.linalg.svd(Q)
        X = Vt[-1, :4]
        X = X[:3] / X[3]
        return X
    
    def Step1(self):
        
        #Step 1: Feature Point Detection, Descriptor, Matching
        S1 = FeatureMatching.FeatureMatching(self.Imgs)
        self.IntPnts = S1.FindIntPnts()
        self.PntsMatch = S1.FindMatches()
    
    def Step2(self):
        
        #Step 2: Calculate F-mat, E-mat, Camera Matrix
        #Assuming camera position for image indexed 0 be [0, 0, 0] in world coordinate
        self.R_t_matrices = [np.hstack((np.eye(3), [[0], [0], [0]]))]
        for i in range(1, self.Imgs.shape[0]):
            points1, points2 = self.get_match_pair_list(0, i)
            S2 = CameraMatrix.FundamentalCameraMatrix()
            S2.ImgPairDetails(points1, points2, self.Imgs[i], self.Imgs[j])
            _, _, _, _, _, self.K, P, _ = S2.Calculate_F_E_C_Matrix(loop_F_mat=self.param_dict['loop_F_mat'],
                                                                    thld_F_inliers=self.param_dict['thld_F_inliers'],
                                                                    thld_P2_angle=self.param_dict['thld_P2_angle'])
            self.R_t_matrices.append(P)
        self.R_t_matrices = np.array(self.R_t_matrices)
        
    def Step3(self):
        
        #Step 3: Find approximate 3D points for 2D correspondences using triangulation
        self.IntPnts3D = []
        for row in self.PntsMatch:
            i, j = np.random.choice(np.where(row != -1)[0], 2, replace=False)
            pi, pj = self.IntPnts[i][row[i]], self.IntPnts[j][row[j]]
            pnt3D = self.triangulate(pi, pj, self.K, self.R_t_matrices[i], self.R_t_matrices[j])
            self.IntPnts3D.append(pnt3D)
        self.IntPnts3D = np.array(self.IntPnts3D)
        
    def Step4(self):
        
        #Step 4: Perform Bundle Adjustmen on IntPnts3D & R_t_matrices
        S4 = BundleAdjustment.BundleAdjustment(self.IntPnts, self.PntsMatch, self.K, self.R_t_matrices, self.IntPnts3D)
        self.R_t_matrices, self.IntPnts3D = S4.PerformBundleAdjustment()
    
    def Step5(self):
        
        #Step 5: Use Dense Matching to generate more points for point cloud
        done = []
        self.NewMatchedPointPairs = {}
        for i in range(self.Imgs.shape[0]):
            for j in range(self.Imgs.shape[0]):
                if i != j and ([i, j] not in done and [j, i] not in done):
                    done.append([i, j])
                    img1 = self.Imgs[i] if self.channels==1 else cv2.cvtColor(self.Imgs[i], cv2.COLOR_BGR2GRAY)
                    img2 = self.Imgs[j] if self.channels==1 else cv2.cvtColor(self.Imgs[j], cv2.COLOR_BGR2GRAY)
                    
                    S5 = DenseMatching.DenseMatching(img1, img2, self.IntPnts[i], self.IntPnts[j])
                    IntPnts1, IntPnts2 = S5.PerformDenseMatching(thld_zncc_seed=self.param_dict['thld_zncc_seed'], 
                                                                w_seed=self.param_dict['w_seed'], 
                                                                thld_zncc_propagation=self.param_dict['thld_zncc_propagation'], 
                                                                thld_confidence=self.param_dict['thld_confidence'], 
                                                                w_propagation=self.param_dict['w_propagation'], 
                                                                N_window=self.param_dict['N_window'], 
                                                                epsilon_propagation=self.param_dict['epsilon_propagation'])
                    
                    self.NewMatchedPointPairs[f'{i} {j}'] = [np.hstack((IntPnts1, np.ones(IntPnts1.shape[0]).T)), 
                                                             np.hstack((IntPnts2, np.ones(IntPnts2.shape[0]).T))]
    
    def Step6(self):
        
        #Step 6: Triangulating NewMatchedPointPairs to get 3D coordinates for Point Cloud
        self.New3DPnts = []
        self.Color3DPnts = []
        for key, val in self.NewMatchedPointPairs.items():
            i, j = list(map(int, key.split()))
            IntPnts1, IntPnts2 = val
            P1, P2 = self.R_t_matrices[i], self.R_t_matrices[j]
            for p1, p2 in zip(IntPnts1, IntPnts2):
                pnt3D = self.triangulate(p1, p2, self.K, P1, P2)
                self.New3DPnts.append([pnt3D, self.Imgs[i][p1[1], p1[0]]])
                
                clr3D = self.Imgs[i][p1[1], p1[0]]
                if type(clr3D) == int:
                    self.Color3DPnts.append([clr3D]*3)
                else:
                    self.Color3DPnts.append(clr3D)
        
        self.New3DPnts = np.array(self.New3DPnts)
        self.Color3DPnts = np.array(self.Color3DPnts)
    
    def Step7(self):
        #Step 7: Visualization
        pcd = open3d.geometry.PointCloud()
        pcd.points = Vector3dVector(self.New3DPnts)
        pcd.colors = Vector3dVector(self.Color3DPnts)
        open3d.visualization.draw_geometries([pcd])
    
    def Reconstruct(self):
        
        #Step 1: Feature Point Detection, Descriptor, Matching
        S1 = FeatureMatching.FeatureMatching(self.Imgs)
        self.IntPnts = S1.FindIntPnts()
        self.PntsMatch = S1.FindMatches()
        
        #Step 2: Calculate F-mat, E-mat, Camera Matrix
        #Assuming camera position for image indexed 2 be [0, 0, 0] in world coordinate
        self.R_t_matrices = [np.hstack((np.eye(3), [[0], [0], [0]]))]
        for i in range(1, self.Imgs.shape[0]): #(img, pnts) in enumerate(zip(self.Imgs[1:], self.IntPnts[1:])):
            points1, points2 = get_match_pair_list(0, i)
            S2 = CameraMatrix.FundamentalCameraMatrix()
            S2.ImgPairDetails(points1, points2, self.Imgs[i], self.Imgs[j])
            _, _, _, _, _, self.K, P, _ = S2.Calculate_F_E_C_Matrix(loop_F_mat=self.param_dict['loop_F_mat'],
                                                                    thld_F_inliers=self.param_dict['thld_F_inliers'],
                                                                    thld_P2_angle=self.param_dict['thld_P2_angle'])
            self.R_t_matrices.append(P)
        self.R_t_matrices = np.array(self.R_t_matrices)
        
        #Step 3: Find approximate 3D points for 2D correspondences using triangulation
        self.IntPnts3D = []
        for row in self.PntsMatch:
            i, j = np.random.choice(np.where(row != -1)[0], 2, replace=False)
            pi, pj = self.IntPnts[i][row[i]], self.IntPnts[j][row[j]]
            pnt3D = self.triangulate(pi, pj, self.K, self.R_t_matrices[i], self.R_t_matrices[j])
            self.IntPnts3D.append(pnt3D)
        self.IntPnts3D = np.array(self.IntPnts3D)
        
        #Step 4: Perform Bundle Adjustmen on IntPnts3D & R_t_matrices
        S4 = BundleAdjustment.BundleAdjustment(self.IntPnts, self.PntsMatch, self.K, self.R_t_matrices, self.IntPnts3D)
        self.R_t_matrices, self.IntPnts3D = S4.PerformBundleAdjustment()
        
        #Step 5: Use Dense Matching to generate more points for point cloud
        done = []
        self.NewMatchedPointPairs = {}
        for i in range(self.Imgs.shape[0]):
            for j in range(self.Imgs.shape[0]):
                if i != j and ([i, j] not in done and [j, i] not in done):
                    done.append([i, j])
                    img1 = self.Imgs[i] if self.channels==1 else cv2.cvtColor(self.Imgs[i], cv2.COLOR_BGR2GRAY)
                    img2 = self.Imgs[j] if self.channels==1 else cv2.cvtColor(self.Imgs[j], cv2.COLOR_BGR2GRAY)
                    
                    S5 = DenseMatching.DenseMatching(img1, img2, self.IntPnts[i], self.IntPnts[j])
                    S5.PerformDenseMatching(thld_zncc_seed=self.param_dict['thld_zncc_seed'], 
                                            w_seed=self.param_dict['w_seed'], 
                                            thld_zncc_propagation=self.param_dict['thld_zncc_propagation'], 
                                            thld_confidence=self.param_dict['thld_confidence'], 
                                            w_propagation=self.param_dict['w_propagation'], 
                                            N_window=self.param_dict['N_window'], 
                                            epsilon_propagation=self.param_dict['epsilon_propagation'])
                    IntPnts1, IntPnts2 = S5.IntPnts
                    
                    self.NewMatchedPointPairs[f'{i} {j}'] = [np.hstack((IntPnts1, np.ones(IntPnts1.shape[0]).T)), 
                                                             np.hstack((IntPnts2, np.ones(IntPnts2.shape[0]).T))]
        
        #Step 6: Triangulating NewMatchedPointPairs to get 3D coordinates for Point Cloud
        self.New3DPnts = []
        self.Color3DPnts = []
        for key, val in self.NewMatchedPointPairs.items():
            i, j = list(map(int, key.split()))
            IntPnts1, IntPnts2 = val
            P1, P2 = self.R_t_matrices[i], self.R_t_matrices[j]
            for p1, p2 in zip(IntPnts1, IntPnts2):
                pnt3D = self.triangulate(p1, p2, self.K, P1, P2)
                self.New3DPnts.append([pnt3D, self.Imgs[i][p1[1], p1[0]]])
                
                clr3D = self.Imgs[i][p1[1], p1[0]]
                if type(clr3D) == int:
                    self.Color3DPnts.append([clr3D]*3)
                else:
                    self.Color3DPnts.append(clr3D)
        
        self.New3DPnts = np.array(self.New3DPnts)
        self.Color3DPnts = np.array(self.Color3DPnts)
        
        #Step 7: Visualization
        pcd = open3d.geometry.PointCloud()
        pcd.points = utility.Vector3dVector(self.New3DPnts)
        pcd.colors = utility.Vector3dVector(self.Color3DPnts)
        open3d.visualization.draw_geometries([pcd])

In [75]:
Construct = Reconstruction(glob.glob('Data/CameraModel/images/ho*')[:3])
Construct.IntPnts = IntPnts
Construct.PntsMatch = PntsMatch

Number of Input Images: 3


In [76]:
Construct.Step1()

NameError: name 'FeatureMatching' is not defined

In [46]:
Imgs[0][25, 20]

131